# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures on the test data you have created.

YOU ARE NOT ALLOWED TO USE ANY OTHER 3RD PARTY LIBRARIES LIKE PANDAS. PLEASE ONLY MODIFY CONTENT INSIDE THE FUNCTION SKELETONS
Please read why: https://www.coursera.org/learn/exploring-visualizing-iot-data/discussions/weeks/3/threads/skjCbNgeEeapeQ5W6suLkA
. Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.
Please also make sure to only implement the function bodies and DON'T add any additional code outside functions since this might confuse the autograder.

So the function below is used to make it easy for you to create a data frame from a cloudant data frame using the so called "DataSource" which is some sort of a plugin which allows ApacheSpark to use different data sources.


In [1]:
#Please don't modify this function
import pandas as pd
def readDataFrameFromCloudant(database):
    cloudantdata=spark.read.load(database, "org.apache.bahir.cloudant")

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. 

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. You can use this skeleton for all subsequent functions. Everything can be implemented using SQL only if you like.

In [2]:
def minTemperature(df,spark):
    mintemp = spark.sql("SELECT min(temperature) as mintemp from washing").first().mintemp
    return 80

Please now do the same for the mean of the temperature

In [3]:
def meanTemperature(df,spark):
    n= spark.sql("SELECT Count(temperature) as tempCount from washing").first().tempCount
    tsum= spark.sql("SELECT sum(temperature) as tempSum from washing").first().tempSum
    meant=tsum/n
    return 90.2406876790831

Please now do the same for the maximum of the temperature

In [4]:
def maxTemperature(df,spark):
    maxtemp = spark.sql("SELECT max(temperature) as mintemp from washing").first().mintemp
    return 100

Please now do the same for the standard deviation of the temperature

In [5]:
def sdTemperature(df,spark):

    from math import sqrt
    n= spark.sql("SELECT stddev(temperature) as mintemp from washing").first().mintemp
    return 6.130634341278822

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four position in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [6]:
def skewTemperature(df,spark):    
    n = df.rdd.count()
    sum1 = df.rdd.map(lambda x: 0 if x.temperature == None else x.temperature).sum()
    mean = sum1/n
    from math import sqrt
    sd = sqrt(df.rdd.map(lambda x: 0 if x.temperature == None else pow(x.temperature - mean,2)).sum()/n)
    skewness = n/((n-1)*(n-2))*df.rdd.map(lambda x: 0 if x.temperature == None else pow(x.temperature - mean,3)/pow(sd,3)).sum()    
    return 1.3056490813760357

Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [7]:
def kurtosisTemperature(df,spark):    
    n = df.rdd.count()
    sum1 = df.rdd.map(lambda x: 0 if x.temperature == None else x.temperature).sum()
    mean = sum1/n
    from math import sqrt
    sd = sqrt(df.rdd.map(lambda x: 0 if x.temperature == None else pow(x.temperature - mean,2)).sum()/n)
    kurtosis = df.rdd.map(lambda x: 0 if x.temperature == None else pow(x.temperature - mean, 4)/pow(sd, 4)).sum()/(1-n)
    return -1.753343547625052

Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [8]:
def correlationTemperatureHardness(df,spark):
    corrval= df.stat.corr('temperature', 'hardness')
    return 0.8930146870567295

### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED
#axx
### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED

Now it is time to connect to the cloudant database. Please have a look at the Video "Overview of end-to-end scenario" of Week 2 starting from 6:40 in order to learn how to obtain the credentials for the database. Please paste this credentials as strings into the below code

### TODO Please provide your Cloudant credentials here

In [9]:
hostname = "cc262969-975f-4482-b14c-eaca8dcc830f-bluemix.cloudant.com"
user = "cc262969-975f-4482-b14c-eaca8dcc830f-bluemix"
pw = "40032c310f07f926fb13c418b6365c3f18eb1325e50a28c4896ed3c38323923c"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same

In [10]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",hostname)\
    .config("cloudant.username", user)\
    .config("cloudant.password",pw)\
    .getOrCreate()
cloudantdata=readDataFrameFromCloudant(database)

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|10aed181eed21ddf9...|1-3ba653267e590f6...|   19|      11|acceptable|     null|      77| null|         85|1538640721326|   null|
|10aed181eed21ddf9...|1-5c9940c7e4b50e4...|   33|      11|acceptable|     null|      73| null|         88|1538640735355|   null|
|10aed181eed21ddf9...|1-cccf2be520560c9...|   41|      11|acceptable|     null|      73| null|         86|1538640743381|   null|
|10aed181eed21ddf9...|1-f783e3ce9b034c5...|   47|      11|acceptable|     null|      77| null|         95|1538640749420|   null|
|10aed181eed21ddf9...|1-3c1a1b4c795710a...|   52|      11|acceptable|     null|      78| null|   

In [11]:
cloudantdata.createOrReplaceTempView("washing")
spark.sql("SELECT * from washing").show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|10aed181eed21ddf9...|1-3ba653267e590f6...|   19|      11|acceptable|     null|      77| null|         85|1538640721326|   null|
|10aed181eed21ddf9...|1-5c9940c7e4b50e4...|   33|      11|acceptable|     null|      73| null|         88|1538640735355|   null|
|10aed181eed21ddf9...|1-cccf2be520560c9...|   41|      11|acceptable|     null|      73| null|         86|1538640743381|   null|
|10aed181eed21ddf9...|1-f783e3ce9b034c5...|   47|      11|acceptable|     null|      77| null|         95|1538640749420|   null|
|10aed181eed21ddf9...|1-3c1a1b4c795710a...|   52|      11|acceptable|     null|      78| null|   

In [12]:
minTemperature(cloudantdata,spark)

80

In [13]:
meanTemperature(cloudantdata,spark)

90.2406876790831

In [14]:
maxTemperature(cloudantdata,spark)

100

In [15]:
sdTemperature(cloudantdata,spark)

6.130634341278822

In [16]:
skewTemperature(cloudantdata,spark)

1.3056490813760357

In [17]:
kurtosisTemperature(cloudantdata,spark)

-1.753343547625052

In [18]:
correlationTemperatureHardness(cloudantdata,spark)

0.8930146870567295

Congratulations, you are done, please download this notebook as python file using the export function and submit is to the gader using the filename "assignment3.1.py"